In [75]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [76]:
df = pd.read_csv("Dataframes/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [77]:
# Verificando se o dataframe possui alguma coluna que tenha algum valor nulo ou faltante
# Como essa dataframe está bem organizado, não precisaremos tratar valores nulos
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [78]:
# Separação das variáveis dependentes e independentes
# X = variáveis dependentes 
# y = variáveis independentes 
X = df.drop('HeartDisease', axis=1)
y = df['HeartDisease']

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    # Separação dos dados de treino e teste (30%) para teste e (70%) para treino
    test_size=0.3,
    random_state=42,
    # Uso do stratify para evitar distorção, garantindo que o modelo aprenda a partir de uma
    # distribuição realista e seja avaliado de forma justa
    stratify=y)

In [80]:
# Features são colunas que são correlacionadas a minha variável Target
# Nosso Target é a coluna 'HeartDisease'
# Nossos Features são todas as colunas restantes do dataframe, excluindo a coluna Target
Features_numericas = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
Features_categoricas = ['Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [81]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Aplicando o ColumnTransformer para padronizar a modificação das colunas, aplicando algoritmos de OneHotEncoder (Para converter colunas categóricas em 0 ou 1)
# E StandardScaler para normalizar os dados númericos no intervalo de 0 e 1.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), Features_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), Features_categoricas)
    ],
    remainder='passthrough'
)

In [82]:
X_train_processado = preprocessor.fit_transform(X_train)
X_test_processado = preprocessor.transform(X_test)

In [83]:
from sklearn.neural_network import MLPClassifier
modelo = MLPClassifier(
    # Duas camadas ocultas (100 e 50 neurônios)
    hidden_layer_sizes=(100,50),
    # Função de ativação para as camadas ocultas
    activation='relu',
    # Otimizador
    solver='adam',
    # Número máximo de épocas (interações) que o modelo vai realizar sobre todo o conjunto de dados de treino
    max_iter=500,
    random_state=42,
    verbose=False
)

In [84]:
#Treino do modelo
print("Iniciando o treino do modelo!")
modelo.fit(X_train_processado,y_train)
print("Modelo treinado com sucesso!")

Iniciando o treino do modelo!
Modelo treinado com sucesso!


In [85]:
y_pred = modelo.predict(X_test_processado)
y_proba = modelo.predict_proba(X_test_processado)[:, 1]

In [86]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
acuracia = accuracy_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)
relatorio = classification_report(y_test,y_pred)

In [87]:
print("\n--- Resultados de Avaliação do MLP ---")
print(f"Acurácia (Accuracy): {acuracia:.4f}")
print(f"AUC-ROC Score: {auc_roc:.4f}")
print("\nRelatório de Classificação (Precision, Recall, F1-Score):")
print(relatorio)


--- Resultados de Avaliação do MLP ---
Acurácia (Accuracy): 0.8768
AUC-ROC Score: 0.9203

Relatório de Classificação (Precision, Recall, F1-Score):
              precision    recall  f1-score   support

           0       0.84      0.89      0.87       123
           1       0.90      0.87      0.89       153

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.88       276
weighted avg       0.88      0.88      0.88       276



## ✅ Análise de Dados de Ataque Cardíaco com MLP

* **Acurácia (Accuracy):** 0.8705 (ou 87.05%): O modelo acertou o diagnóstico de doença cardíaca (ou a ausência dela) em cerca de 87% dos pacientes no conjunto de teste.

* **AUC-ROC Score: 0.9122:** Este valor, próximo de 1, indica que o modelo tem uma excelente capacidade de discriminar (separar) pacientes com doença cardíaca (classe 1) e pacientes sem a doença (classe 0).

---

## 🔎 Interpretação das Métricas:
1. **Precisão (Classe 1 - Doença Cardíaca): 0.85**
    * Dos pacientes que o modelo **previu** que teriam doença cardíaca (1), 85% estavam realmente corretos.
2. **Recall (Classe 1 - Doença Cardíaca): 0.89**
    * Dos pacientes que **realmente** tinham doença cardíaca (1), o modelo conseguiu **identificar corretamente** 89% deles. Este é um valor importante em diagnósticos, pois indica a capacidade do modelo de não deixar passar um caso positivo (baixo número de **Falsos Negativos**).
3. **F1-Score:**
    * O F1-Score (0.87 para ambas as classes) é a média harmônica de Precisão e Recall. Ele indica um bom equilíbrio entre as duas métricas, mostrando que o modelo é robusto tanto em sua capacidade de acerto quanto de cobertura.